<a href="https://colab.research.google.com/github/murmayasa/murmayasa/blob/main/Prediction_Of_Heart_Disease_Using_A_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [21]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn.metrics import mean_squared_error
keras.optimizers.Adam

keras.optimizers.adam.Adam

# Mengakses Dataset

In [22]:
from google.colab import drive
drive.mount('/content/drive/')
csv_path = '/content/drive/MyDrive/Colab Notebooks/Heart_Disease_Prediction.csv'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Membaca Dataset

In [23]:
data = pd.read_csv(csv_path)

# Memilih Kolom

In [24]:
columns = ['Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120', 'EKG results', 'Max HR',
           'Exercise angina', 'ST depression', 'Slope of ST', 'Number of vessels fluro', 'Thallium', 'Heart Disease']
data = data[columns]

# Memisahkan fitur dan label

In [25]:
X = data.drop('Heart Disease', axis=1)
y = data['Heart Disease']

# Normalisasi fitur menggunakan StandardScaler

In [26]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi dataset menjadi data latih dan data uji

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Mengubah data menjadi bentuk yang sesuai untuk model RNN

In [68]:
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

label_encoder = LabelEncoder()

# Mengubah nilai dalam kolom target menjadi representasi numerik
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Pembuatan Model

In [69]:
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[1]), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Fungsi untuk menghitung MAPE

In [70]:
def calculate_mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Compile Model

In [81]:
learning_rate = 0.005
model.compile(loss='binary_crossentropy',
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 64)                19968     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 20,033
Trainable params: 20,033
Non-trainable params: 0
_________________________________________________________________


# Melatih model

In [82]:
model.fit(X_train_rnn, y_train, epochs=50, batch_size=32, validation_data=(X_test_rnn, y_test))
print("*** Hasil Training ***")
loss, accuracy = model.evaluate(X_test_rnn, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/50
6/6 [==============================] - 3s 82ms/step - loss: 0.2807 - accuracy: 0.8889 - val_loss: 0.3649 - val_accuracy: 0.8395
Epoch 2/50
6/6 [==============================] - 0s 19ms/step - loss: 0.2641 - accuracy: 0.8889 - val_loss: 0.3746 - val_accuracy: 0.8272
Epoch 3/50
6/6 [==============================] - 0s 19ms/step - loss: 0.2562 - accuracy: 0.8995 - val_loss: 0.3786 - val_accuracy: 0.8272
Epoch 4/50
6/6 [==============================] - 0s 19ms/step - loss: 0.2494 - accuracy: 0.8995 - val_loss: 0.3870 - val_accuracy: 0.8148
Epoch 5/50
6/6 [==============================] - 0s 19ms/step - loss: 0.2408 - accuracy: 0.9048 - val_loss: 0.3882 - val_accuracy: 0.8148
Epoch 6/50
6/6 [==============================] - 0s 20ms/step - loss: 0.2332 - accuracy: 0.9101 - val_loss: 0.3930 - val_accuracy: 0.8148
Epoch 7/50
6/6 [==============================] - 0s 18ms/step - loss: 0.2255 - accuracy: 0.9101 - val_loss: 0.3998 - val_accuracy: 0.8148
Epoch 8/50
6/6 [===========

# Evaluasi model

In [83]:
loss, accuracy = model.evaluate(X_test_rnn, y_test)
print("Loss: ", loss)
print("Accuracy: ", accuracy)
y_pred = model.predict(X_test_rnn)

3/3 [==============================] - 0s 5ms/step - loss: 0.7415 - accuracy: 0.7778
Loss:  0.7415006160736084
Accuracy:  0.7777777910232544
3/3 [==============================] - 0s 3ms/step


# Tahap Denormalisasi

In [84]:
y_pred_classes = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))
y_pred_denormalized = scaler.inverse_transform(y_pred.reshape(-1, 1) * np.sqrt(scaler.var_) + scaler.mean_)
y_test_denormalized = scaler.inverse_transform(y_test.reshape(-1, 1) * np.sqrt(scaler.var_) + scaler.mean_)

# Menghitung RMSE & MAPE

In [85]:
rmse = np.sqrt(np.mean((y_test_denormalized - y_pred_denormalized)**2))
mape = np.mean(np.abs((y_test_denormalized - y_pred_denormalized) / y_test_denormalized)) * 100

print("RMSE:", rmse)
print("MAPE:", mape)

RMSE: 322.63473884844313
MAPE: 6.605605107808382
